In [1]:
import numpy as np

In [2]:
# first, define our problem size
n = 10
alpha = np.random.random((n, 1))
k = np.random.random((n, n))
sigma = np.array([[k[i][i]] for i in range(n)])

# TODO: figure out how to stitch together the r_opt values with the actual stock names
stock_names = [f'stock_{i}' for i in range(n)]

# TODO? figure out how to generate the confidence intervals of how much money you expect to make from this

# TODO, pick one stock to be the baseline (stock_0). Figure out some way of comparing
# stock_0's expected performance compared to our optimal solver

# Katrina's draft:

max_num_stocks = 5

# DRAFT END

# goal, generate some vector r such that:
# sum(r) = 1.0
# r represents our optimal investment portfolio


In [3]:
numbers = [1,2,3,4]
letters = ['a', 'b', 'c', 'd']

# zipping lists
print(list(zip(numbers, letters)))

# enumerating lists
print(list(enumerate(numbers)))

# for each syntax
print([(num, letter) for num, letter in zip(numbers, letters)])

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]
[(0, 1), (1, 2), (2, 3), (3, 4)]
[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


In [4]:
# element-wise math

# square every term in alpha
alpha_element_squared = alpha * alpha
sigma_element_squared = sigma * sigma

# the 3.9 formula from
# https://maslov.bioengineering.illinois.edu/optimal_investment_ijtaf.pdf
alpha / (alpha_element_squared + sigma_element_squared)


array([[0.90963365],
       [0.36922373],
       [1.005612  ],
       [1.08644939],
       [0.59274884],
       [1.08724341],
       [0.29006334],
       [0.80155065],
       [1.64340393],
       [0.53849331]])

In [5]:
# invert matrices
k_inv = np.linalg.inv(k)

# formula 3.10 from 
# https://maslov.bioengineering.illinois.edu/optimal_investment_ijtaf.pdf
r_opt = np.dot(k_inv, alpha)


In [6]:
# hints that I'm not sure how they all go together

# from formula 3.11
# you know that if \lambda >= \alpha_i, then we can ignore that term

# Maybe if you sort all fo the \alpha_i terms, you can see if a solution for lambda exists where all of the smaller
# values are ignored, and all of the larger values are effected by \lambda

# if you look at each term in the sum of 3.12, they are piecewise linear wrt. \lambda


In [ ]:
# idea one to solve this: use line search
# main idea: function 3.12 is monotically decreasing
# so if we pick upper/lower bounds of lambda, we can just do a line search and call it a day
# monotonically decreasing means the derivative of the function wrt lambda is <= 0 across all values of lambda

# upper bound, we know that if lambda causes all of the terms to be zero, we went too far.
# i.e., alpha_i - upper_bound_lambda <= 0 \forall i
upper_bound_lambda = np.max(alpha)

# lower bound, we know that each term in the sum is >= 0 and <= the total sum term.
# so just find the lower bound of each term and take the maximum out of all of these terms
# for each term, lambda = alpha_i - alpha_i^2 - sigma_i^2
# I know this because I can do high-school algebra in my head... work it out on paper if you can't
lower_bound_lambda = np.max(alpha)
# # # TODO, think about cases where there are zero values in r_opt
# # r_opt_top_stocks = r_opt[np.argsort(-r_opt)[:max_num_stocks]]
# # r_opt_top_stocks /= np.sum(r_opt_top_stocks)

# r_opt_top_stocks = [r_opt_with_names[best_stock_index][0] for best_stock_index in best_stock_indices]
# r_opt_top_stocks = np.array(r_opt_top_stocks_with_names) / np.sum(r_opt_top_stocks_with_names)
# r_opt_top_stocks

# r_opt_top_stocks_with_names = [r_opt_with_names[best_stock_index] for best_stock_index in best_stock_indices]
# r_opt_top_stocks_with_namespha - alpha_element_squared - sigma_element_squared)

print(lower_bound_lambda, upper_bound_lambda)

# now line search/binary search... first we need to evaluate the value of the 3.12 summation expression
def eval_expression(lambda_value):
    # subtract one so I can get something like \sum - 1 = 0 to solve.
    fractional_terms = (alpha - lambda_value) / (alpha_element_squared + sigma_element_squared)
    # heavyside function
    fractional_terms = np.maximum(fractional_terms, 0)
    return np.sum(fractional_terms) - 1

# sanity check, the correct value (0) is between the upper bound and lower bound
eval_expression(lower_bound_lambda), eval_expression(upper_bound_lambda)

# there's a tolerance term usually, more on that later
tolerance = 1e-9

# initialize our loop with our initial guess
lower_lambda = lower_bound_lambda
upper_lambda = upper_bound_lambda
middle_lambda = (lower_bound_lambda + upper_bound_lambda) / 2

iterations = 0
while np.abs(eval_expression(middle_lambda)) > tolerance:
    middle_value = eval_expression(middle_lambda)
    if middle_value > 0:
        lower_lambda = middle_lambda
    else:
        upper_lambda = middle_lambda
    middle_lambda = (lower_lambda + upper_lambda) / 2

    iterations += 1
    print(f'iteration: {iterations}, {middle_lambda}, {eval_expression(middle_lambda)}')


0.9928702178639255 0.9928702178639255
iteration: 1, 0.9928702178639255, -1.0
iteration: 2, 0.9928702178639255, -1.0
iteration: 3, 0.9928702178639255, -1.0
iteration: 4, 0.9928702178639255, -1.0
iteration: 5, 0.9928702178639255, -1.0
iteration: 6, 0.9928702178639255, -1.0
iteration: 7, 0.9928702178639255, -1.0
iteration: 8, 0.9928702178639255, -1.0
iteration: 9, 0.9928702178639255, -1.0
iteration: 10, 0.9928702178639255, -1.0
iteration: 11, 0.9928702178639255, -1.0
iteration: 12, 0.9928702178639255, -1.0
iteration: 13, 0.9928702178639255, -1.0
iteration: 14, 0.9928702178639255, -1.0
iteration: 15, 0.9928702178639255, -1.0
iteration: 16, 0.9928702178639255, -1.0
iteration: 17, 0.9928702178639255, -1.0
iteration: 18, 0.9928702178639255, -1.0
iteration: 19, 0.9928702178639255, -1.0
iteration: 20, 0.9928702178639255, -1.0
iteration: 21, 0.9928702178639255, -1.0
iteration: 22, 0.9928702178639255, -1.0
iteration: 23, 0.9928702178639255, -1.0
iteration: 24, 0.9928702178639255, -1.0
iteration: 

iteration: 211, 0.9928702178639255, -1.0
iteration: 212, 0.9928702178639255, -1.0
iteration: 213, 0.9928702178639255, -1.0
iteration: 214, 0.9928702178639255, -1.0
iteration: 215, 0.9928702178639255, -1.0
iteration: 216, 0.9928702178639255, -1.0
iteration: 217, 0.9928702178639255, -1.0
iteration: 218, 0.9928702178639255, -1.0
iteration: 219, 0.9928702178639255, -1.0
iteration: 220, 0.9928702178639255, -1.0
iteration: 221, 0.9928702178639255, -1.0
iteration: 222, 0.9928702178639255, -1.0
iteration: 223, 0.9928702178639255, -1.0
iteration: 224, 0.9928702178639255, -1.0
iteration: 225, 0.9928702178639255, -1.0
iteration: 226, 0.9928702178639255, -1.0
iteration: 227, 0.9928702178639255, -1.0
iteration: 228, 0.9928702178639255, -1.0
iteration: 229, 0.9928702178639255, -1.0
iteration: 230, 0.9928702178639255, -1.0
iteration: 231, 0.9928702178639255, -1.0
iteration: 232, 0.9928702178639255, -1.0
iteration: 233, 0.9928702178639255, -1.0
iteration: 234, 0.9928702178639255, -1.0
iteration: 235, 

iteration: 553, 0.9928702178639255, -1.0
iteration: 554, 0.9928702178639255, -1.0
iteration: 555, 0.9928702178639255, -1.0
iteration: 556, 0.9928702178639255, -1.0
iteration: 557, 0.9928702178639255, -1.0
iteration: 558, 0.9928702178639255, -1.0
iteration: 559, 0.9928702178639255, -1.0
iteration: 560, 0.9928702178639255, -1.0
iteration: 561, 0.9928702178639255, -1.0
iteration: 562, 0.9928702178639255, -1.0
iteration: 563, 0.9928702178639255, -1.0
iteration: 564, 0.9928702178639255, -1.0
iteration: 565, 0.9928702178639255, -1.0
iteration: 566, 0.9928702178639255, -1.0
iteration: 567, 0.9928702178639255, -1.0
iteration: 568, 0.9928702178639255, -1.0
iteration: 569, 0.9928702178639255, -1.0
iteration: 570, 0.9928702178639255, -1.0
iteration: 571, 0.9928702178639255, -1.0
iteration: 572, 0.9928702178639255, -1.0
iteration: 573, 0.9928702178639255, -1.0
iteration: 574, 0.9928702178639255, -1.0
iteration: 575, 0.9928702178639255, -1.0
iteration: 576, 0.9928702178639255, -1.0
iteration: 577, 

iteration: 1100, 0.9928702178639255, -1.0
iteration: 1101, 0.9928702178639255, -1.0
iteration: 1102, 0.9928702178639255, -1.0
iteration: 1103, 0.9928702178639255, -1.0
iteration: 1104, 0.9928702178639255, -1.0
iteration: 1105, 0.9928702178639255, -1.0
iteration: 1106, 0.9928702178639255, -1.0
iteration: 1107, 0.9928702178639255, -1.0
iteration: 1108, 0.9928702178639255, -1.0
iteration: 1109, 0.9928702178639255, -1.0
iteration: 1110, 0.9928702178639255, -1.0
iteration: 1111, 0.9928702178639255, -1.0
iteration: 1112, 0.9928702178639255, -1.0
iteration: 1113, 0.9928702178639255, -1.0
iteration: 1114, 0.9928702178639255, -1.0
iteration: 1115, 0.9928702178639255, -1.0
iteration: 1116, 0.9928702178639255, -1.0
iteration: 1117, 0.9928702178639255, -1.0
iteration: 1118, 0.9928702178639255, -1.0
iteration: 1119, 0.9928702178639255, -1.0
iteration: 1120, 0.9928702178639255, -1.0
iteration: 1121, 0.9928702178639255, -1.0
iteration: 1122, 0.9928702178639255, -1.0
iteration: 1123, 0.992870217863925

iteration: 1771, 0.9928702178639255, -1.0
iteration: 1772, 0.9928702178639255, -1.0
iteration: 1773, 0.9928702178639255, -1.0
iteration: 1774, 0.9928702178639255, -1.0
iteration: 1775, 0.9928702178639255, -1.0
iteration: 1776, 0.9928702178639255, -1.0
iteration: 1777, 0.9928702178639255, -1.0
iteration: 1778, 0.9928702178639255, -1.0
iteration: 1779, 0.9928702178639255, -1.0
iteration: 1780, 0.9928702178639255, -1.0
iteration: 1781, 0.9928702178639255, -1.0
iteration: 1782, 0.9928702178639255, -1.0
iteration: 1783, 0.9928702178639255, -1.0
iteration: 1784, 0.9928702178639255, -1.0
iteration: 1785, 0.9928702178639255, -1.0
iteration: 1786, 0.9928702178639255, -1.0
iteration: 1787, 0.9928702178639255, -1.0
iteration: 1788, 0.9928702178639255, -1.0
iteration: 1789, 0.9928702178639255, -1.0
iteration: 1790, 0.9928702178639255, -1.0
iteration: 1791, 0.9928702178639255, -1.0
iteration: 1792, 0.9928702178639255, -1.0
iteration: 1793, 0.9928702178639255, -1.0
iteration: 1794, 0.992870217863925

...

iteration: 10798, 0.9928702178639255, -1.0
iteration: 10799, 0.9928702178639255, -1.0
iteration: 10800, 0.9928702178639255, -1.0
iteration: 10801, 0.9928702178639255, -1.0
iteration: 10802, 0.9928702178639255, -1.0
iteration: 10803, 0.9928702178639255, -1.0
iteration: 10804, 0.9928702178639255, -1.0
iteration: 10805, 0.9928702178639255, -1.0
iteration: 10806, 0.9928702178639255, -1.0
iteration: 10807, 0.9928702178639255, -1.0
iteration: 10808, 0.9928702178639255, -1.0
iteration: 10809, 0.9928702178639255, -1.0
iteration: 10810, 0.9928702178639255, -1.0
iteration: 10811, 0.9928702178639255, -1.0
iteration: 10812, 0.9928702178639255, -1.0
iteration: 10813, 0.9928702178639255, -1.0
iteration: 10814, 0.9928702178639255, -1.0
iteration: 10815, 0.9928702178639255, -1.0
iteration: 10816, 0.9928702178639255, -1.0
iteration: 10817, 0.9928702178639255, -1.0
iteration: 10818, 0.9928702178639255, -1.0
iteration: 10819, 0.9928702178639255, -1.0
iteration: 10820, 0.9928702178639255, -1.0
iteration: 

iteration: 11054, 0.9928702178639255, -1.0
iteration: 11055, 0.9928702178639255, -1.0
iteration: 11056, 0.9928702178639255, -1.0
iteration: 11057, 0.9928702178639255, -1.0
iteration: 11058, 0.9928702178639255, -1.0
iteration: 11059, 0.9928702178639255, -1.0
iteration: 11060, 0.9928702178639255, -1.0
iteration: 11061, 0.9928702178639255, -1.0
iteration: 11062, 0.9928702178639255, -1.0
iteration: 11063, 0.9928702178639255, -1.0
iteration: 11064, 0.9928702178639255, -1.0
iteration: 11065, 0.9928702178639255, -1.0
iteration: 11066, 0.9928702178639255, -1.0
iteration: 11067, 0.9928702178639255, -1.0
iteration: 11068, 0.9928702178639255, -1.0
iteration: 11069, 0.9928702178639255, -1.0
iteration: 11070, 0.9928702178639255, -1.0
iteration: 11071, 0.9928702178639255, -1.0
iteration: 11072, 0.9928702178639255, -1.0
iteration: 11073, 0.9928702178639255, -1.0
iteration: 11074, 0.9928702178639255, -1.0
iteration: 11075, 0.9928702178639255, -1.0
iteration: 11076, 0.9928702178639255, -1.0
iteration: 

iteration: 11283, 0.9928702178639255, -1.0
iteration: 11284, 0.9928702178639255, -1.0
iteration: 11285, 0.9928702178639255, -1.0
iteration: 11286, 0.9928702178639255, -1.0
iteration: 11287, 0.9928702178639255, -1.0
iteration: 11288, 0.9928702178639255, -1.0
iteration: 11289, 0.9928702178639255, -1.0
iteration: 11290, 0.9928702178639255, -1.0
iteration: 11291, 0.9928702178639255, -1.0
iteration: 11292, 0.9928702178639255, -1.0
iteration: 11293, 0.9928702178639255, -1.0
iteration: 11294, 0.9928702178639255, -1.0
iteration: 11295, 0.9928702178639255, -1.0
iteration: 11296, 0.9928702178639255, -1.0
iteration: 11297, 0.9928702178639255, -1.0
iteration: 11298, 0.9928702178639255, -1.0
iteration: 11299, 0.9928702178639255, -1.0
iteration: 11300, 0.9928702178639255, -1.0
iteration: 11301, 0.9928702178639255, -1.0
iteration: 11302, 0.9928702178639255, -1.0
iteration: 11303, 0.9928702178639255, -1.0
iteration: 11304, 0.9928702178639255, -1.0
iteration: 11305, 0.9928702178639255, -1.0
iteration: 

iteration: 11606, 0.9928702178639255, -1.0
iteration: 11607, 0.9928702178639255, -1.0
iteration: 11608, 0.9928702178639255, -1.0
iteration: 11609, 0.9928702178639255, -1.0
iteration: 11610, 0.9928702178639255, -1.0
iteration: 11611, 0.9928702178639255, -1.0
iteration: 11612, 0.9928702178639255, -1.0
iteration: 11613, 0.9928702178639255, -1.0
iteration: 11614, 0.9928702178639255, -1.0
iteration: 11615, 0.9928702178639255, -1.0
iteration: 11616, 0.9928702178639255, -1.0
iteration: 11617, 0.9928702178639255, -1.0
iteration: 11618, 0.9928702178639255, -1.0
iteration: 11619, 0.9928702178639255, -1.0
iteration: 11620, 0.9928702178639255, -1.0
iteration: 11621, 0.9928702178639255, -1.0
iteration: 11622, 0.9928702178639255, -1.0
iteration: 11623, 0.9928702178639255, -1.0
iteration: 11624, 0.9928702178639255, -1.0
iteration: 11625, 0.9928702178639255, -1.0
iteration: 11626, 0.9928702178639255, -1.0
iteration: 11627, 0.9928702178639255, -1.0
iteration: 11628, 0.9928702178639255, -1.0
iteration: 

iteration: 11972, 0.9928702178639255, -1.0
iteration: 11973, 0.9928702178639255, -1.0
iteration: 11974, 0.9928702178639255, -1.0
iteration: 11975, 0.9928702178639255, -1.0
iteration: 11976, 0.9928702178639255, -1.0
iteration: 11977, 0.9928702178639255, -1.0
iteration: 11978, 0.9928702178639255, -1.0
iteration: 11979, 0.9928702178639255, -1.0
iteration: 11980, 0.9928702178639255, -1.0
iteration: 11981, 0.9928702178639255, -1.0
iteration: 11982, 0.9928702178639255, -1.0
iteration: 11983, 0.9928702178639255, -1.0
iteration: 11984, 0.9928702178639255, -1.0
iteration: 11985, 0.9928702178639255, -1.0
iteration: 11986, 0.9928702178639255, -1.0
iteration: 11987, 0.9928702178639255, -1.0
iteration: 11988, 0.9928702178639255, -1.0
iteration: 11989, 0.9928702178639255, -1.0
iteration: 11990, 0.9928702178639255, -1.0
iteration: 11991, 0.9928702178639255, -1.0
iteration: 11992, 0.9928702178639255, -1.0
iteration: 11993, 0.9928702178639255, -1.0
iteration: 11994, 0.9928702178639255, -1.0
iteration: 

In [ ]:
import matplotlib.pyplot as plt

x = np.linspace(0, 1, 101)
plt.plot(x, [eval_expression(el) for el in x], label='f(lambda)')
plt.plot([0, 1], [0, 0], label='0')
plt.legend()

plt.show()


In [ ]:
# get the optimal investment portfolio
r_opt = np.maximum(0, (alpha - middle_lambda) / (alpha_element_squared + sigma_element_squared))
r_opt = r_opt.flatten()

# r_opt_with_names = list(zip(r_opt, stock_names))
best_stock_indices = np.argsort(-r_opt)[:max_num_stocks]
best_stock_prices = r_opt[best_stock_indices]
best_stock_prices /= np.sum(best_stock_prices)

r_opt_top_stocks_with_names = []
for best_stock_price, best_stock_index in zip(best_stock_prices, best_stock_indices):
    r_opt_top_stocks_with_names.append([best_stock_price, stock_names[best_stock_index]])
r_opt_top_stocks_with_names

In [20]:
# brief lecture on probabilities (mean/variance)
# E[X]=mean(X), Var[X]=Variance(X)
# let a be a constant
# Var[X] = E[X^2] - E[X]^2
# aE[X] = E[aX]
# a^2Var[X] = Var[(aX)]
# Let v be some vector, K be the covariance matrix
# Var(Kv) = v^t K v

# we now have r_opt, find the mean and variance of investing in r_opt
# alpha, sigma

# Amber and Katrina's draft:

import numpy as np

totals = [100]

nums = []
for i in totals:
    if i == 0: 
        nums.append([0 for i in range(9)])
        continue
    total = i
    temp = []
    for i in range(9):
        val = np.random.randint(0, total)
        temp.append(val)
        total -= val
    temp.append(total)
    nums.append(temp)

dec = np.array(nums)/100

# print(dec)

r_opt = dec


n = 10

alpha = np.random.random((n, 1))

r_opt = r_opt[0]

# E[X]: assuming this means how much money we'll make total using r_opt portfolio proportions, multiplied by 
# the expected return of each individual stock

# expected_val_array = np.multiply(r_opt, alpha)
expected_val_array = [r_opt[i]*alpha[i] for i in range(len(r_opt))] 
expected_val = sum(expected_val_array)/len(expected_val_array)

print(expected_val_array)
print(expected_val)

variance = [sum((expected_val_array[i] - expected_val)**2 for i in range(len(expected_val_array) - 1))] / len(expected_val_array)

# standard_dev = np.sqrt(variance)

print(variance)
# print(standard_dev)

# end of draft




[array([0.84066882]), array([0.00310566]), array([0.]), array([0.01255559]), array([0.00110552]), array([0.00783669]), array([0.01001077]), array([0.]), array([0.]), array([0.0029272])]
[0.08782103]


TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [ ]:
r_opt_top_stocks

In [ ]:
np.sum(r_opt_top_stocks)

In [ ]:
sum([el[0] for el in r_opt_top_stocks_with_names])

In [ ]:
r_opt_top_stocks_with_names

In [ ]:
x = 2
x /= 2
print(x)

In [ ]:
numbers = [1,2,3,4]
letters = ['a', 'b', 'c', 'd']

# zipping lists
print(list(zip(numbers, letters)))

# enumerating lists
print(list(enumerate(numbers)))

# for each syntax
print([(num, letter) for num, letter in zip(numbers, letters)])

In [ ]:
np.argsort(-r_opt)[:max_num_stocks]

In [ ]:
# how would we print the amount we should invest in each stock by name?

# what variables should you look at to compute this?
# r_opt, stock_names

# what should the output look like?
# [[stock_0, investment_percent_0], [stock_1, investment_percent_1], etc...]

# how do we get there?
list(zip(r_opt, stock_names))


In [ ]:
r_opt.shape[0] == n, len(stock_names)

In [ ]:
r_opt_top_stocks

In [ ]:
# sorts things by index
np.argsort(-r_opt.flatten())

In [ ]:
r_opt

In [ ]:
r_opt[7], np.max(r_opt)